In [ ]:
# 1. Încărcarea și curățarea datelor: Vom citi fișierul JSON și
# vom extrage datele relevante. De asemenea, vom gestiona datele
# lipsă și vom transforma diverse tipuri de date într-un format
# potrivit pentru antrenare.

# 2. Explorarea datelor: Vom analiza distribuția variabilelor și 
# vom identifica eventualele outliers și dezechilibre în date.

# 3. Feature engineering: Vom crea caracteristici (features) 
# utile pentru antrenarea modelului, evitând pe cât posibil 
# folosirea textului din review-uri în această etapă inițială.

# 4. Antrenarea și evaluarea modelului: Vom folosi metode de 
# machine learning și deep learning pentru a antrena 
# modele și a evalua performanța acestora.

# 5. Optimizarea modelului: Vom ajusta parametrii și vom 
# experimenta cu diferite modele pentru a îmbunătăți 
# acuratețea predicțiilor.

# Partea de cod începe acum:
# 1. Încărcarea și curățarea datelor. Începem prin a citi și a
# curăță din fișierul JSON:
import os
import json
import pandas as pd

# Citim datele din fișierul JSON
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as finp:
        data = json.load(fin)
    return data

file_path = 'games.json'
data = load_data(file_path)

# Transformăm datele într-un DataFrame pentru o analiză mai ușoară
games_list = []

for appID, game in data.items():
    game_info = {
        'appID': appID,
        'name': game['name'],
        'releaseDate': game['release_date'],
        'estimatedOwners': game['estimated_owners'],
        'peakCCU': game['peak_ccu'],
        'required_age': game['required_age'],
        'price': game['price'],
        'dlcCount': game['dlc_count'],
        'longDesc': game['detailed_description'],
        'shortDesc': game['short_description'],
        'languages': game['supported_languages'],
        'fullAudioLanguages': game['full_audio_languages'],
        'reviews': game['reviews'],
        'headerImage': game['header_image'],
        'website': game['website'],
        'supportWeb': game['support_url'],
        'supportEmail': game['support_email'],
        'supportWindows': game['windows'],
        'supportMac': game['mac'],
        'supportLinux': game['linux'],
        'metacriticScore': game['metacritic_score'],
        'metacriticURL': game['metacritic_url'],
        'userScore': game['user_score'],
        'positive': game['positive'],
        'negative': game['negative'],
        'scoreRank': game['score_rank'],
        'achievements': game['achievements'],
        'recommendations': game['recommendations'],
        'notes': game['notes'],
        'averagePlaytime': game['average_playtime_forever'],
        'averageplaytime2W': game['average_playtime_2weeks'],
        'medianPlaytime': game['median_playtime_forever'],
        'medianPlaytime2W': game['median_playtime_2weeks'],
        'developers': game['developers'],
        'publishers': game['publishers'],
        'categories': game['categories'],
        'genres': game['genres'],
        'screenshots': game['scrennshots'],
        'movies': game['movies'],
        'tags': game['tags']
    }
    games_list.append(game_info)
    
game_df = pd.DataFrame(games_list)

# Vizualizăm primele rânduri ale DataFrame-ului
print(games_df.head())

# 2. Explorarea datelor
# Vom analiza distribuția datelor și vom vizualiza relațiile 
# dintre variabile pentru a înțelege mai bine structura setului de date:
import matplotlib.pyplot as plt
import seaborn as sns

# Conversia coloanelor numerice la tipul potrivit
games_df['price'] = pd.to_numeric(games_df['price'], errors='coerce')
games_df['peakCCU'] = pd.to_numeric(games_df['peak_ccu'], errors='coerce')
games_df['required_age'] = pd.to_numeric(games_df['required_age'], errors='coerce')
games_df['metacriticScore'] = pd.to_numeric(games_df['metacritic_score'], errors='coerce')
games_df['userScore'] = pd.to_numeric(games_df['user_score'], errors='coerce')
games_df['positive'] = pd.to_numeric(games_df['positive'], errors='coerce')
games_df['negative'] = pd.to_numeric(games_df['negative'], errors='coerce')
games_df['achievements'] = pd.to_numeric(games_df['achievements'], errors='coerce')
games_df['recommendations'] = pd.to_numeric(games_df['recommendations'], errors='coerce')
games_df['averagePlaytime'] = pd.to_numeric(games_df['average_playtime_forever'], errors='coerce')
games_df['averageplaytime2W'] = pd.to_numeric(games_df['average_playtime_2weeks'], errors='coerce')
games_df['medianPlaytime'] = pd.to_numeric(games_df['median_playtime_forever'], errors='coerce')
games_df['medianPlaytime2W'] = pd.to_numeric(games_df['median_playtime_2weeks'], errors='coerce')
games_df['dlcCount'] = pd.to_numeric(games_df['dlc_count'], errors='coerce')

# Vizualizarea distribuției prețurilor
plt.figure(figsize=(10,6))
sns.histplot(games_df['price'], bins=50, kde=True)
plt.title('Distribuția prețurilor jocurilor')
plt.xlabel('Preț (USD)')
plt.ylabel('Frecvența')
plt.show()

# Vizualizarea relației dintre preț și numărul de jucători concurenți
plt.figure(figsize=(10,6))
sns.scatterplot(data=games_df, x='peakCCU',y='price')
plt.title('Relația dintre numărul de jucători concurenți și preț')
plt.xlabel('Numărul de jucători concurenți(peakCCU)')
plt.ylabel('Preț (USD)')
ply.show()

# 3 Feature engineering
# Vom transforma datele textuale și categorice în format 
# numeric, folosind tehnici precum one-hot encoding pentru 
# limbile suportate și genuri, și vom trata datele lipsă.
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Codificarea limbilor suportate
languages = games_df['languages'].str.get_dummies(sep=',')
games_df = pd.concat([games_df,languages], axis=1)

# Codificarea genurilor
genres = games_df['genres'].str.get_dummies(sep=',')
games_df = pd.concat([games_df, genres], axis=1)

# Tratamentul valorilor lipsă
imputer = SimpleImputer(strategy='mean')
games_df['price'] = imputer.fit_transform(games_df[['price']])

# Selectarea caracteristicilor relevante pentru antrenare
feature = games_df[['peakCCU', 'required_age', 'dlcCount', 'metacriticScore', 'userScore', 'positive', 'negative', 'achievements', 'recommendations', 'averagePlaytime', 'averageplaytime2W', 'medianPlaytime', 'medianPlaytime2W'] + list(languages.columns) + list(genres.columns)]
target = games_df['price']

# 4. Antrenarea și evaluarea modelului
# Vom folosi metode de machine learning pentru a antrena un 
# model și vom evalua performanța acestuia.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Împărțirea datelor în seturi de antrenament și testare
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Antrenarea unui model RandomForest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicția pe setul de testare
y_pred = model.predict(X_test)

# Evaluarea performanței modelului
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'Root Mean Squared Error: {rmse}')

# 5. Optimizarea modelului
# Vom ajusta parametrii și vom experimenta cu diferite 
# modele pentru a îmbunătăți performanța.

from sklearn.model_selection import GridSearchCV

# Optimizarea modelului RandomForest folosind GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=model,






